In [1]:
import geopandas as gpd
import pandas as pd
from shapely import Point, LineString
import osmnx as ox
from shapely.geometry import box, Polygon, MultiPolygon
from shapely.ops import transform
from functools import partial
import pyproj
import time

In [2]:
# Import MSOA lookup
msoas = gpd.read_file('data/MSOA_EngWal_Dec_2011_Generalised_ClippedEW_0/Middle_Layer_Super_Output_Areas_December_2011_Generalised_Clipped_Boundaries_in_England_and_Wales.shp').to_crs(4326).set_index('msoa11cd')

# Import MSOA 2011 OD data
od_data = pd.read_parquet('data/od_2011.parquet')

#Import LSOAs
lsoas = gpd.read_file('data/LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4_6029841263726194941.gpkg').to_crs(4326)
lsoas = pd.concat([lsoas, lsoas.bounds], axis=1)

#Import lsoa to msoa look up
lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')

/tmp/ipykernel_35260/2575286281.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')


In [8]:
# Read in 2016

t0 = time.time()
G2016 = gpd.read_file('osm2016.geojson')
t1 = time.time()
print('Read geojson : {}'.format(t1 - t0))

t0 = time.time()
def extract_coordinates(line_string, line_index, coord_index):
    #line = LineString(eval(line_string))
    return list(line_string.coords)[line_index][coord_index]

# Apply the function to the DataFrame column
G2016['coordinates_u_x'] = G2016['geometry'].apply(extract_coordinates,line_index=0,coord_index=0)
G2016['coordinates_u_y'] = G2016['geometry'].apply(extract_coordinates,line_index=0,coord_index=1)

G2016['coordinates_v_x'] = G2016['geometry'].apply(extract_coordinates,line_index=-1,coord_index=0)
G2016['coordinates_v_y'] = G2016['geometry'].apply(extract_coordinates,line_index=-1,coord_index=1)

node_set_1 = G2016[['node1','coordinates_u_x','coordinates_u_y']].drop_duplicates(subset='node1')
node_set_2 = G2016[['node2','coordinates_v_x','coordinates_v_y']].drop_duplicates(subset='node2')

node_set_1 = node_set_1.rename(columns={'node1': 'osmid','coordinates_u_x': 'x','coordinates_u_y': 'y'})
node_set_2 = node_set_2.rename(columns={'node2': 'osmid','coordinates_v_x': 'x','coordinates_v_y': 'y'})

node_df = pd.concat([node_set_1,node_set_2]).drop_duplicates(subset='osmid').set_index('osmid')
t1 = time.time()
print('Read Get Nodes : {}'.format(t1 - t0))

del(node_set_1)
del(node_set_2)

t0 = time.time()
G2016['key'] = 0
edge_gdf = gpd.GeoDataFrame(G2016[['node1','node2','key','geometry','length','lts']].set_index(['node1','node2','key']),geometry=G2016[['node1','node2','key','geometry','length','lts']].set_index(['node1','node2','key'])['geometry'])
t1 = time.time()
print('Read Get Edges : {}'.format(t1 - t0))

del(G2016)

t0 = time.time()
G = ox.graph_from_gdfs(node_df,edge_gdf,{'crs': 'epsg:4326'})
t1 = time.time()
print('Construct Network : {}'.format(t1 - t0))

Read geojson : 521.9917948246002
Read Get Nodes : 156.08479642868042
Read Get Edges : 9.695680141448975
Construct Network : 88.07392525672913


In [ ]:
nodes = ox.graph_to_gdfs(G, edges=False)

In [9]:
bbx_expansion = 0.5

#Below function from ChatGPT
#Get expanded network - method 1 km buffer
def expand_bbox(original_bbox, expansion_distance_km=5):
    # Create a Shapely geometry object for the original bounding box
    original_geometry = box(*original_bbox)
    # Define a function to project the geometry to a new coordinate reference system
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),  # WGS 84 coordinate reference system
        pyproj.Proj(proj='utm', zone=33, ellps='WGS84')  # Example: UTM Zone 33
    )
    # Project the original geometry to the new coordinate reference system
    projected_geometry = transform(project, original_geometry)
    # Calculate the expansion distance in the projected coordinate system
    expansion_distance_meters = expansion_distance_km * 1000
    # Expand the geometry by the specified distance
    expanded_geometry = projected_geometry.buffer(expansion_distance_meters)
    # Project the expanded geometry back to the original coordinate reference system
    expanded_geometry = transform(partial(pyproj.transform, pyproj.Proj(proj='utm', zone=33, ellps='WGS84'), pyproj.Proj(init='epsg:4326')), expanded_geometry)
    # Get the coordinates of the expanded bounding box
    expanded_bbox = expanded_geometry.bounds
    return expanded_bbox, expanded_geometry

def create_bounding_box(geometry1, geometry2):

    # Calculate the union of all polygons in each multipolygon
    union_geometry1 = geometry1.convex_hull
    union_geometry2 = geometry2.convex_hull
    # Calculate the union of the convex hulls of the two multipolygons
    union_geometry = union_geometry1.union(union_geometry2)
    # Get the bounding box of the union geometry
    bounding_box = union_geometry.bounds
    return bounding_box

def get_centrality(edges):

    edge_list = []
    for i,e in edges.iterrows():
        edge_list.append(tuple([str(i[0]),str(i[1]),e['length'],e['LTS']]))
        
    g = gt.Graph()

    elength = g.new_ep("float")
    elts = g.new_ep("int")

    g.add_edge_list(edge_list, hashed = True,eprops=[elength, elts])

    vp, ep = gt.betweenness(g,weight=elength)

    lts0 = []
    lts1 = []
    lts2 = []
    lts3 = []
    lts4 = []

    for e in g.edges():
        if elts[e] == 0:
            lts0.append(ep[e])
        elif elts[e] == 1:
            lts1.append(ep[e])
        elif elts[e] == 2:
            lts2.append(ep[e])
        elif elts[e] == 3:
            lts3.append(ep[e])
        elif elts[e] == 4:
            lts4.append(ep[e])

    return lts0,lts1,lts2,lts3,lts4

In [10]:
# Import MSOA lookup
msoas = gpd.read_file('data/MSOA_EngWal_Dec_2011_Generalised_ClippedEW_0/Middle_Layer_Super_Output_Areas_December_2011_Generalised_Clipped_Boundaries_in_England_and_Wales.shp').to_crs(4326).set_index('msoa11cd')

# Import MSOA 2011 OD data
od_data = pd.read_parquet('data/od_2011.parquet')

#Import LSOAs
lsoas = gpd.read_file('data/LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4_6029841263726194941.gpkg').to_crs(4326)
lsoas = pd.concat([lsoas, lsoas.bounds], axis=1)

#Import lsoa to msoa look up
lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')

#York LSOAs
york_model = pd.read_csv('data/LSOA_york_model.csv')
york_lsoas = list(york_model['LSOA_code'])

/tmp/ipykernel_1025/1176549301.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')


In [11]:
#for lsoa_id in york_lsoas:

lsoa_id = york_lsoas[0]

#lsoa = lsoas[lsoas['LSOA11CD'] == test_lsoa['LSOA11CD'].values[0]]
lsoa_lookup = lookup[lookup['LSOA11CD'] == lsoa_id][:1]
lsoa = lsoas[lsoas['LSOA11CD'] == lsoa_lookup['LSOA11CD'].values[0]]

In [15]:
lsoa_bbox = box(lsoa['maxy'],lsoa['miny'],lsoa['minx'],lsoa['maxx'])

/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/geometry/polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]


In [16]:
intersecting_nodes = nodes[nodes.intersects(lsoa_bbox)].index
G_bb = G.subgraph(intersecting_nodes)

In [21]:
expanded_bbox, expanded_geometry = expand_bbox((lsoa['minx'], lsoa['miny'], lsoa['maxx'], lsoa['maxy']), expansion_distance_km=bbx_expansion)
intersecting_nodes = nodes[nodes.intersects(box(expanded_bbox[0],expanded_bbox[1],expanded_bbox[2],expanded_bbox[3]))].index
G_bb_exp = G.subgraph(intersecting_nodes)

/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/geometry/polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/chris/miniconda3/envs/ate/lib

In [22]:
bike_ods = od_data[(od_data['geo_code1'] == lsoa_lookup['MSOA11CD'].values[0]) & (od_data['bicycle'] > 0)][['geo_code2','bicycle']].set_index('geo_code2')
bike_ods['geometry'] = msoas['geometry']
bike_ods = bike_ods.dropna()

origin_geom = lsoa['geometry'].values[0]
destination_geom = msoas.loc[bike_ods['bicycle'].idxmax()]['geometry']

bounding_box_od = create_bounding_box(origin_geom, destination_geom)

intersecting_nodes = nodes[nodes.intersects(box(bounding_box_od[0],bounding_box_od[1],bounding_box_od[2],bounding_box_od[3]))].index
G_bb_od = G.subgraph(intersecting_nodes)